**Notebook 2 – Artificial Neural Networks for neuroimaging with Keras**

_This notebook tries to learn and predict labels from fMRI data._
jens.schwarzbach@ukr.de

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/jvschw/ml4ni/blob/master/ML4NI/2_neural_nets_for_neuroimaging_data_with_keras_v2.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

run this cell only in google colab (uncomment before)

In [ ]:
! git clone https://github.com/jvschw/ml4ni.git
! python3 -m pip install -U nilearn

# Artificial Neural Networks in Neuroimaging

## Forward and Reverse Inference

Can we read somebody’s thoughts with fMRI and machine learning?

Typically, we use fMRI for producing *Statistical Parameter Maps* that indicate which voxels show activity that can be better explained by a model than by chance. We use massive univariate (i.e. voxel wise) statistics to map an explanatory variable (the experimental design) to observed data (voxel time-courses). This process is called **forward inference**. In decoding or **reverse inference** we try to find the mapping from observed data (patterns of activity or some SPMs) to an explanatory variable (scores or labels). Reverse inference is exactly what we want to achieve in fMRI-based mind reading: We want to find out what went on in the participant’s mind (labels) based on patterns of brain activity.

<img src="https://github.com/jvschw/ml4ni/blob/master/ML4NI/images/ni/forward_and_reverse_inference.png?raw=1">

**Figure 1.** *In encoding or forward inference we use massive univariate (i.e. voxel wise) statistics to map an explanatory variable (the experimental design) to observed data (voxel time-courses). This addresses the question of which voxels show activity that can be better explained by a particular model than by chance. In decoding or reverse inference we try to find the mapping from observed data (patterns of activity) to an explanatory variable (scores or labels).* 

## Task and data

Here, we will start with brain- or mind reading by creating a model to predict from brain activity what the participant was *doing*. Then we will use that model to predict what the participant was *thinking of doing*. 

We conducted an fMRI experiment with 10 runs of a foot-finger-tongue mapper and one run of motor imagery. In runs 1-10 we asked participants to move particular body parts (left index finger, right index finger, left toe, right toe, tongue in left cheek, tongue in right cheek). In run 11 we asked participants to imagine performing the respective actions.

<img src="https://github.com/jvschw/ml4ni/blob/master/ML4NI/images/ni/fft_design.png?raw=1">

**Figure 2.** *Schematic of an experimental run. Each bar depicts a time-period of 16s, with the color indicating the task (red: wiggle the left toe, yellow: wiggle the right tow, green: move the left index finger up and down, cyan: move the right index finger up and down, blue: move the tong inside the left cheek, purple: move the tongue inside the right cheek, black: pause).*

**Part** I of our exercise is to perform the forward inference:  Starting from the participant’s action or imagination (e.g. moving the left index finger) we want to find out which brain areas activate. In **part II**, which is about reverse inference, we want to build and train a classifier that can predict the experimental conditions in data like this. Finally, in **part III** on cross-decoding we will test whether a model that has been trained on action execution can predict the labels of the eleventh run when the participant only imagined the movements.

# Part I: Forward inference (estimate a model of brain activity resulting from experimental manipulations)

How to perform forward inference is handled in several fMRI classes. You can find excellent tutorials at https://fsl.fmrib.ox.ac.uk/fslcourse/.
In a nutshell: With fMRI we measure the blood oxygenation level dependent (BOLD) response of the brain, which we take as a measure of brain activity. The 4D-images that we obtain can be regarded as many timeseries that are arranged in 3D. The figure below shows us the timeseries (in red) of one voxel (volumetric pixel). In fMRI data analysis we test to which level a set of processes (predictors), which should produce timeseries that are timelocked to occurences of these processes (multiple humps in the two blue lines), can explain the data better than a null-hypothesis (that these processes do not matter in a given voxel). 

We can formulate this problem as a regression problem in the context of the General Linear Model (GLM):
\begin{align}
y = \beta X + e
\end{align}
with y being the to-be-explained timecourse (your observation), X being the design matrix (your hypothesis: which process -encoded in the columns- happened when -encoded in the rows-), \$\beta\$ being a vector with one weight per predictor (this is what you want to find out: how high should each \$\beta\$ be such that the particular model explains the data best?), and e being the error that your model cannot explain.

In massive univariate testing we calculate one set of beta values for each voxel, and we create as many beta maps as we have predictors. If you devide beta by the standard deviation of the error (one per voxel), you get a t-map. This is what we will be working with: a set of t-maps (one for each experimental condition).

<img src="https://github.com/jvschw/ml4ni/blob/master/ML4NI/images/ni/mass_univariate_GLM_small.png?raw=1">

**Figure 3.** Simplified workflow of a univariate GLM-based fMRI analysis. Observed timecourse y (red), design matrix X with predictors x1 and x2 (blue) and error (black). This particular GLM aims at explaining y with βX, i.e. $β_{1}$$x_{1}$ + $β_{2}$$x_{2}$  by finding those β-weights that minimize the squared error (of y-βX).


Let's explore some data. We have a low-resolution structural (T1-weighted) scan and we have a statistical parameter map from the first run that contains the t-values for the statistical contrast of moving the left index finger vs baseline (FING_L).
Using the nibabel (https://nipy.org/nibabel/) library we can read data in a host of imaging formats, here compressed nifti. 

In [ ]:
import nibabel as nib #if this throws an error consider 'python3 -m pip install -U nibabel'

#Load structural scan
example_anat = '/content/ml4ni/ML4NI/parameterMaps/FFT001_T1_lores.nii.gz'
#example_anat = './parameterMaps/FFT001_T1_lores.nii.gz'

imgT1 = nib.load(example_anat)

#load statistical parameter map for one run
example_func = '/content/ml4ni/ML4NI/parameterMaps/FFT001_R01_TSTAT1.nii.gz'
#example_func = './parameterMaps/FFT001_R01_TSTAT1.nii.gz'
imgSPM = nib.load(example_func)

With nilearn (https://nilearn.github.io), a library for machine learning projects targeted at neuroimaging, we can visualize the data.

In [ ]:
from nilearn import plotting
from nilearn.image import mean_img
plotting.view_img(imgSPM, bg_img=imgT1, threshold=2.5)

**Figure 4.** Rendering of statistical parameter map of one subject and one run in which the participant moved the left index finger. The background is a low-resolution rendering of that participant's structural scan. [click and move the mouse] 

From our mapping experiment described above we have created one nifti-file (FFT001.nii.gz) that contains 66 t-maps, which are the t-maps of all 11 runs with each run producing 6 maps, one per condition ("FingL", "FingR", "FootL", "FootR", "TongueL", "TongueR"). In the next parts we will use these t-maps as features for machine learning and the ordered list of experimental conditions as labels. Let's load the whole stack of 3D maps and extract all features into a big 4D matrix (space x number_of_maps).

In [ ]:
#fNameMaps = './parameterMaps/FFT001.nii.gz'
fNameMaps = '/content/ml4ni/ML4NI/parameterMaps/FFT001.nii.gz'
imgSPM = nib.load(fNameMaps)

#extract features (t-values)
X4D = imgSPM.get_fdata()
print("Data dimensionality (xDim, yDim, zDim, noOfMaps) is:", X4D.shape) #(xDim, yDim, zDim, noOfMaps)

Each of our 66 SPMs (11 runs with 6 stimulation-blocks per run) is 3D [64, 100, 67] (voxels in the x-direction right to left, voxels in the y-direction front to back, and voxels in the z-direction top to bottom). For machine learning we want to flatten them into vectors.

In [ ]:
import numpy as np
xDim=X4D.shape[0];
yDim=X4D.shape[1];
zDim=X4D.shape[2];
numberOfMaps=X4D.shape[3];
Xflat = np.empty([numberOfMaps, xDim*yDim*zDim])
counter = 0
for iMap in range(numberOfMaps):
    thisMap = X4D[:, :, :,iMap]
    Xflat[iMap,:] = thisMap.flatten()
print('Dimensionality of feature map Xflat', Xflat.shape)

Now we are ready to visualize all the features that we are going to use for predicting the labels or targets (brain states). Do you think that all features are informative?

In [ ]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(32, 8))
ax = fig.add_subplot(111)
hImage = ax.imshow(Xflat, cmap=mpl.cm.hsv)
ax.set_aspect(aspect=2000)
ax.set_xlabel('features');
ax.set_ylabel('maps');
fig.colorbar(hImage);

**Figure 5.** 2D-heatmap of t-values for the entire experiment. Rows 0-5 refer to "FingL", "FingR", "FootL", "FootR", "TongueL", "TongueR", repectively for run 1. This arrangement is repeated for all runs, yielding 66 maps with 428800 features each.

And here is how we create our targets (I actually prefer calling targets 'labels'). We will use the experimental conditions as targets. The stacked maps are organized in 11 (number of runs) x 6 (number of conditions) in a fixed sequence. We will code conditions as targets.

In [ ]:
targets = np.empty([66, 1], dtype=int) #condition number

counter=0
for iRun in range(11):
    for iCond in range(6):
        targets[counter] = iCond #0-5
        counter+=1
class_names = ["FingL", "FingR", "FootL", "FootR", "TongueL", "TongueR"]
print("targets: ", targets.transpose()) #to save screen real estate, transpose for printing
print('target-dimensions: ', targets.shape)

# Part II: Reverse inference (predict label, i.e. mental state, from patterns of brain activity)

Now we have everything we need for reverse inference, i.e. for predicting the label (or mental state) from features (patterns of brain activity).

## The general strategy of training a model

Our goal is to train a model on existing data that will generalize well on new data. But how can you tell whether your model will generalize well on new data? You cannot know, because new data will a) come in the future and b) you will not know exactly what the new data will be (if you knew, there wouldn't be much predicting to do, right?). In the meantime you can try to *estimate* how well the model will generalize. 
When we evaluate an algorithm, we are in fact evaluating all steps in the procedure, including how the training data was prepared (e.g. scaling), the choice of algorithm (e.g. deep learning), and how the chosen algorithm was configured (e.g. number_of_hidden_layers=3).
The performance measure calculated on the predictions is an estimate of the skill of the whole procedure.
We generalize the performance measure from:
“the skill of the procedure on the test set“
to
“the skill of the procedure on unseen data“.
To this aim you take your existing data and split it. Here are two splitting strategies that depend on whether you have a lot of labeled data or not.  

*If you have a lot of data*, your first split is into Training Data and Test Data (see also Figure 6 below). Then you use further splits on your training data for k-fold cross validation in order to finetune so-called hyperparameters (number of hidden layers, number of neurons per layer, learning rate, activation functions and so forth). Try to find the best combination of hyperparameters by training on part of the data (white) and estimate on unseen data (yellow). You will get k accuracies, one for each split. The average and standard deviation of these k accuracies will be your estimator for how well the model generalizes (the average) and how confident you are with your estimate (the spread).

Since this estimate is biased due to the fact that you have used all of your data during cross validation, you can now generate an unbiased estimate of your accuracy (but not of the spread) by retraining your model on all the training data and predicting the (known) labels of the test dataset. If the prediction accuracy on the test dataset is similar to the estimated accuracy, you have good reasons to assume that a) your estimate was good, and b) that your model did not overfit the training data. If your prediction accuracy on the test data is much lower than its estimate, it is quite likely that your model did overfit the training data or that you made a mistake. However, do not tune your model based on knowledge about the test data.  

*If you do **not** have a lot of data*, you may not be able to afford the first split into training and test data. You will use all of your data for training. Like in the procedure above, you can use cross validation to estimate how well your model will generalize to unseen data. However, you have no chance of checking this, because you do not have any test data. But if your resampling procedure was done properly, this value will only be slightly biased. Now it is time to retrain your final model on all the data you have available. This procedure leads to biased (inflated) expectations about generalizability but it  may be better than training your model on an even smaller dataset and actually performing worse.
***Since we do not have a lot of data in this example, we take this latter approach here.***
<img src="https://github.com/jvschw/ml4ni/blob/master/ML4NI/images/ni/crossvalidation_jvs_small.png?raw=1">

**Figure 6.** Data-splitting for training and evaluating a model. **Training Dataset:** The sample of data used to fit the model. **Validation Dataset:** The sample of data used to provide an unbiased evaluation of a model fit on the training dataset while tuning model hyperparameters. The evaluation becomes more biased as skill on the validation dataset is incorporated into the model configuration.
**Test Dataset:** The sample of data used to provide an unbiased evaluation of a final model fit on the training dataset. **Notes:** The validation dataset may also play a role in other forms of model preparation, such as feature selection. The final model could be fit on the aggregate of the training and validation datasets.

Find below a small example on crossvalidation

In [ ]:
import numpy as np
from sklearn.model_selection import KFold

X = np.array(["a", "b", "c", "d", "e", "f", "g", "h", "i", "j"])
kf = KFold(n_splits=10) #try with 5
for train, test in kf.split(X):
    print("%s %s" % (train, test))

for train, test in kf.split(X):
    print("%s %s" % (X[train], X[test]))



## The general strategy of crossvalidation in Keras

Below there is an outline of how to implement a crossvalidation approach with Keras (adapted from https://datascience.stackexchange.com/questions/11747/cross-validation-in-keras). This code is meant as a preview of the steps that follow when we fill these different functions with code.

**NOTE**: executing the cell below should produce errors because this is incomplete code only meant to provide an overview

In [ ]:
#!!!DO NOT EXECUTE!!! 

from sklearn.model_selection import StratifiedKFold

def load_data():
    # load your data using this function

def create model():
    # create your model using this function

def train_and_evaluate__model(model, data_train, labels_train, data_test, labels_test):
    # fit and evaluate here.
    #model.fit...


#MAIN PROGRAM
#set number of desired folds
n_folds = 10

#load data
data, labels, header_info = load_data()

#create shuffled, representative folds
skf = StratifiedKFold(labels, n_folds=n_folds, shuffle=True)

#loop over folds
#     create model
#     train and evaluate model
for i, (train, test) in enumerate(skf):
    print "Running Fold", i+1, "/", n_folds
    model = None # Clearing the NN.
    model = create_model()
    train_and_evaluate_model(model, data[train], labels[train], data[test], labels[test])

## Implementation of crossvalidation for fMRI execution-mapper in Keras

### Prerequisites
This section imports the necessary libraries and defines dome utility functions for plotting.

In [ ]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

# TensorFlow ≥2.0 is required
import tensorflow as tf
assert tf.__version__ >= "2.0"

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=18)
mpl.rc('xtick', labelsize=16)
mpl.rc('ytick', labelsize=16)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "ni"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")
import tensorflow as tf
from tensorflow import keras

### Loading the data

We will use two datasets: 1.) a structural scan which we use for feature selection (use brain-voxels only), and 2) a statistical parameter maps (SPMs) from 10 runs with 6 conditions each of a Foot-Finger-Tongue movement mapper. Each run's maps are ordered alphabetically ("FingL", "FingR", "FootL", "FootR", "TongueL", "TongueR").

The SPMs are stacked 3D-maps ([x, y, z, numberOfMaps]). We flatten them to one vector per map (with as many elements as there are voxels [1, x `*`y`*`z]). Then we mask them, i.e. we only select elements that are located in the brain ([1, numberOfBrainVoxels]). Finally, we stack these vectors such that we obtain a matrix (nMaps=60,nFeatures=numberOfBrainVoxels).

In [ ]:
import nibabel as nib #if this throws an error consider 'python3 -m pip install -U nibabel'

def load_data():
    #load a structural scan and use brain voxels as mask for statistical parameter maps (SPMs)
    
    #Load structural scan
    ref_anat = '/content/ml4ni/ML4NI/parameterMaps/FFT001_T1_lores.nii.gz'
    #ref_anat = './parameterMaps/FFT001_T1_lores.nii.gz'
    imgT1 = nib.load(ref_anat)
    
    #extract features (voxel intensities of the structural scan)
    XT1 = imgT1.get_fdata() 
    
    #create mask from structural scan where voxels have a value higher than 0
    msk2 = XT1>0 
    nMaskedVoxels = msk2.sum()
    
    #Load SPMs (stacked 3D maps)
    fNameSPM = '/content/ml4ni/ML4NI/parameterMaps/FFT001.nii.gz'
    #fNameSPM = './parameterMaps/FFT001.nii.gz'
    imgSPM = nib.load(fNameSPM)
    
    #extract features (t-values)
    XSPM = imgSPM.get_fdata() 
    print("Data dimensionality is:", XSPM.shape)
    nMaps=XSPM.shape[3]
    
    #create flattened and masked feature matrix[nMaps,nFeaturesInsideBrain]
    XflatMasked = np.empty([nMaps, nMaskedVoxels])
    for iMap in range(nMaps):
        #extract one flattened map
        tmpFlat = XSPM[:, :, :,iMap].flatten()
        #mask flattened map and assign to feature matrix
        XflatMasked[iMap,:]=tmpFlat[XT1.flatten()>0] 
    print("Dimensionality of flattened and masked feature matrix: ", XflatMasked.shape)    
    
    #SPLIT EXECUTION DATA (MAPS 1:60) FROM IMAGERY DATA (MAPS 61-66)
    #first 60 maps 10 runs with 6 conditions each
    dataLearn=XflatMasked[:60] 
    #last 6 maps (run 11 with 6 conditions). This is not really a holdout map, since this
    #is imagery data, whereas the first 60 maps are execution data
    dataHoldout=XflatMasked[60:] 
    
    #CREATE LABELS
    #labels
    #chunks = np.empty([66, 1], dtype=int) #run number [not yet used]
    labels = np.empty([nMaps, 1], dtype=int) #condition number
    counter=0
    for iRun in range(11):
        for iCond in range(6):
            #chunks[counter] = iRun #11 runs (0-10)
            labels[counter] = iCond #6 conditions (0-5)
            counter+=1
    labelsLearn=labels[:60] #first 60 (0:59) execution
    labelsHoldout=labels[60:] #last 6 (60:65) imagery
    
    class_names=["FingL", "FingR", "FootL", "FootR", "TongueL", "TongueR"]  
    
    return dataLearn, labelsLearn, dataHoldout, labelsHoldout, class_names, msk2;

### Creating a model

Next comes a function that can create keras models with a variable number of hidden layers, number of neurons per layer, learning rate, and number of input features for us. This function takes up to 4 input parameters, for which we also define default values.

In [ ]:
def create_model(n_hidden=2, n_neurons=30, learning_rate=3e-3, nFeatures=100):
    # create your model using this function
    print(n_hidden, n_neurons, learning_rate)
    model = keras.models.Sequential()   
    model.add(keras.layers.Flatten(input_shape=(nFeatures,)))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu"))
    model.add(keras.layers.Dense(6, activation="softmax"))
    optimizer = keras.optimizers.SGD(lr=learning_rate)
    model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    return model;

### Test and evaluate a model

Here, we create a function that tests and evaluates a model. This function also saves the fitted model for each fold, which will come in handy for later exercises.

In [ ]:
import time

def train_and_evaluate_model(model, data_train, labels_train, data_test, labels_test, fold):
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")+"fft_keras_model"+'{:04d}'.format(fold)
    run_logdir=os.path.join('./my_logs', run_id)

    #----------------------------------------------------------
    #callback functions, which will be executed during training
    #----------------------------------------------------------
    #send data to tensorboard, for online monitoring the training process 
    tensorboard_cb = keras.callbacks.TensorBoard(run_logdir,histogram_freq=1)
    #checkpoint callbacks save the model in a particular state, here the best model (lowest validation-loss)
    checkpoint_cb = keras.callbacks.ModelCheckpoint("fft_keras_model"+'{:04d}'.format(fold)+".h5", save_best_only=True)
    #early stopping is a method to avoid overfitting (here defined as 10 epochs without reducing validation-loss)
    early_stopping_cb = keras.callbacks.EarlyStopping(patience=10,
                                                  restore_best_weights=True)

    #----------------------------------------------------------
    #model fitting
    #----------------------------------------------------------
    history = model.fit(data_train, labels_train, epochs=20, batch_size=6,
                    validation_data=(data_test, labels_test),
                    callbacks=[checkpoint_cb, early_stopping_cb, tensorboard_cb])
    
    #----------------------------------------------------------
    #model evaluation (performance measures and graphs)
    #----------------------------------------------------------
    print('Model evaluation ')
    modEval=model.evaluate(data_test,labels_test) 
    import pandas as pd
    pd.DataFrame(history.history).plot(figsize=(8, 5))
    plt.grid(False)
    plt.gca().set_ylim(0, 1)
    #save_fig("keras_learning_curves_plot")
    plt.show()
    
    return modEval, history;

### Main program

Learn to predict different executed movements using 10-fold cross validation

In [ ]:
from sklearn.model_selection import KFold
#----------------------------------------------------------
#number of folds (should be number of runs)
#----------------------------------------------------------
n_folds=10

#----------------------------------------------------------
#load data
#----------------------------------------------------------
dataLearn, labelsLearn, dataHoldout, labelsHoldout, class_names, msk2 = load_data()
nFeatures=dataLearn.shape[1]

#----------------------------------------------------------
#TURN ON TENSORBOARD
#----------------------------------------------------------
#PROBABLY BETTER TO COMMENT THIS OUT WHEN RUNNING N COLAB
#%load_ext tensorboard
#%tensorboard --logdir=./my_logs --port=7007  #--port=6006

# Clear any logs from previous runs
!rm -rf ./my_logs/ 

#----------------------------------------------------------
#Train and evaluate model with k-fold cross validation
#----------------------------------------------------------
skf = KFold(n_folds).split(dataLearn)
history=[]
modEval=[]
for i, (train, test) in enumerate(skf):
    print("Running Fold", i+1, "/", n_folds)
    model = None # Clearing the NN.
    model = create_model(3, 100, 3e-3, nFeatures)
    e,h=train_and_evaluate_model(model, dataLearn[train], labelsLearn[train],
                                 dataLearn[test],labelsLearn[test], i)
    modEval.append(e)
    history.append(h)
    print(train)

#----------------------------------------------------------
#we can estimate how well our model should perform on new data
#by taking the mean and standard deviation or actually standard error
#for loss and accuracy from our 10 folds
#----------------------------------------------------------
avgValMean = np.mean(modEval, axis=0)
avgValStd = np.std(modEval, axis=0)
print('Average Validation Loss    ', '{:5.3f}'.format(avgValMean[0]), ', se=', '{:5.3f}'.format(avgValStd[0]/np.sqrt(n_folds)))
print('Average Validation Accuracy', '{:5.3f}'.format(avgValMean[1]), ', se=', '{:5.3f}'.format(avgValStd[1]/np.sqrt(n_folds)))


Now we know to which degree our model should be able to generalize to new, unseen data of the same kind. If we wanted to sell this model as a brain reader for executed actions, we could advertise it with the following performance parameters.  

In [ ]:
print('Show me your brain and I tell you what you are doing!')
print('Estimated Loss    ', '{:5.3f}'.format(avgValMean[0]), ', se=', '{:5.3f}'.format(avgValStd[0]/np.sqrt(n_folds)))
print('Estimated Accuracy', '{:5.3f}'.format(avgValMean[1]), ', se=', '{:5.3f}'.format(avgValStd[1]/np.sqrt(n_folds)))

Over ninety percent accuracy to be expected. Not bad! But is that good? This may depend on the application. But remember, we can make such accurate predictions on the single subject level!
By the way, we have not yet created our final model. All of the above was to estimate how good the final model will be. Creating the final model is as simple as training it on our available data from 10 runs.

In [ ]:
#create model
model = create_model(3, 100, 3e-3, nFeatures)

#fit model with early stopping (which I don't think will become effective)
#early_stopping_cb = keras.callbacks.EarlyStopping(monitor='loss', patience=5,
#                                                  restore_best_weights=True)

finalHistory = model.fit(dataLearn, labelsLearn, epochs=20, batch_size=6)

#save model
model.save("FFT_execution_final.h5")

#show how the model converges
import pandas as pd
pd.DataFrame(finalHistory.history).plot(figsize=(8, 5))
plt.grid(False)
plt.gca().set_ylim(0, 1)
plt.show()


# Part III. When we train on executing movements, can we predict which movement the participant imagines?

Such an approach is called cross-decoding. It is easier than you might think. When loading the data, we created a holdout dataset from the last 6 statistical parameter maps. Let's just evaluate our model on these data.

In [ ]:
model.evaluate(dataHoldout, labelsHoldout)

Well, 33-50%. However, remember that this is 2-3 times the probability to guess this right, which is 1/6 (or 16.66%). So which imagined movements don't we really get?

In [ ]:
def annotated_confusion_matrix(cm, class_names):
    fig, ax = plt.subplots()
    im = ax.imshow(cm)

    # We want to show all ticks...
    ax.set_xticks(np.arange(len(class_names)))
    ax.set_yticks(np.arange(len(class_names)))
    # ... and label them with the respective list entries
    ax.set_xticklabels(class_names)
    ax.set_yticklabels(class_names)
    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor");
    ax.set_ylabel('predicted')
    ax.set_xlabel('exp. condition')


    # Loop over data dimensions and create text annotations.
    for i in range(len(class_names)):
        for j in range(len(class_names)):
            text = ax.text(j, i, cm[i, j],
                       ha="center", va="center", color="w")



In [ ]:
from sklearn.metrics import confusion_matrix
y_predHoldout = model.predict_classes(dataHoldout)
print("labels: ", labelsHoldout)
print("predictions: ", y_predHoldout)

cm = confusion_matrix(labelsHoldout, y_predHoldout)
annotated_confusion_matrix(cm, class_names)

How to read this? Columns are the truth (experimental conditions), rows are the predictions. 

Are there particular tasks our model is bad at predicting?










# Sources and Credits

Much of the code is inspired by or taken from Aurelien Geron's excellent book "Hands-On Machine Learning with Scikit-Learn and TensorFlow" and its corresponding jupyter notebooks https://github.com/ageron/handson-ml2. 

Section 2 draws upon Jenkinson, Bijsterbosch, Chappell, & Winkler's "Short introduction to the General Linear Model for Neuroimaging" https://www.fmrib.ox.ac.uk/primers/appendices/glm.pdf.

Section 3.1 is adapted from Jason Brownlee's blog on machine learning https://machinelearningmastery.com/train-final-machine-learning-model/.